In [1]:
from urllib2 import Request, urlopen, HTTPError
from urlparse import urlunparse, urlparse
import json 
import pandas as pd
from matplotlib import pyplot as plt
import requests

In [2]:
from bs4 import BeautifulSoup
import urllib

<b>Step 1</b> Get the links of different app categories on iTunes.

In [4]:
r = urllib.urlopen('https://itunes.apple.com/us/genre/ios-books/id6018?mt=8').read()
soup = BeautifulSoup(r)
print type(soup)

<class 'bs4.BeautifulSoup'>


In [5]:
all_categories = soup.find_all("div", class_="nav")
category_url = all_categories[0].find_all(class_ = "top-level-genre")
categories_url = pd.DataFrame()
for itm in category_url:
    category = itm.get_text()
    url = itm.attrs['href']
    d = {'category':[category], 'url':[url]}
    df = pd.DataFrame(d)
    categories_url = categories_url.append(df, ignore_index = True)
print categories_url

                  category                                                url
0                    Books  https://itunes.apple.com/us/genre/ios-books/id...
1                 Business  https://itunes.apple.com/us/genre/ios-business...
2                 Catalogs  https://itunes.apple.com/us/genre/ios-catalogs...
3                Education  https://itunes.apple.com/us/genre/ios-educatio...
4            Entertainment  https://itunes.apple.com/us/genre/ios-entertai...
5                  Finance  https://itunes.apple.com/us/genre/ios-finance/...
6             Food & Drink  https://itunes.apple.com/us/genre/ios-food-dri...
7                    Games  https://itunes.apple.com/us/genre/ios-games/id...
8         Health & Fitness  https://itunes.apple.com/us/genre/ios-health-f...
9                Lifestyle  https://itunes.apple.com/us/genre/ios-lifestyl...
10  Magazines & Newspapers  https://itunes.apple.com/us/genre/ios-magazine...
11                 Medical  https://itunes.apple.com/us/genre/io

In [6]:
categories_url['url'][0]

'https://itunes.apple.com/us/genre/ios-books/id6018?mt=8'

<b>Step2</b>
Get the links for all popular apps of different catigories on iTunes.

In [7]:
def extract_apps(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r)
    apps = soup.find_all("div", class_="column")
    apps_link = apps[0].find_all('a')
    column_first = pd.DataFrame()
    for itm in apps_link:
        app_name = itm.get_text()
        url = itm.attrs['href']
        d = {'category':[app_name], 'url':[url]}
        df = pd.DataFrame(d)
        column_first = column_first.append(df, ignore_index = True)
    apps_link2 = apps[1].find_all('a')
    column_second = pd.DataFrame()
    for itm in apps_link2:
        app_name = itm.get_text()
        url = itm.attrs['href']
        d = {'category':[app_name], 'url':[url]}
        df = pd.DataFrame(d)
        column_second = column_second.append(df, ignore_index = True)
    apps_link3 = apps[2].find_all('a')
    column_last = pd.DataFrame()
    for itm in apps_link3:
        app_name = itm.get_text()
        url = itm.attrs['href']
        d = {'category':[app_name], 'url':[url]}
        df = pd.DataFrame(d)
        column_last = column_last.append(df, ignore_index = True)
    Final_app_link = pd.DataFrame()
    Final_app_link = Final_app_link.append(column_first, ignore_index = True)
    Final_app_link = Final_app_link.append(column_second, ignore_index = True)
    Final_app_link = Final_app_link.append(column_last, ignore_index = True)
    return Final_app_link

In [8]:
app_url = pd.DataFrame()
for itm in categories_url['url']:
    apps = extract_apps(itm)
    app_url = app_url.append(apps, ignore_index = True)

In [9]:
app_url['url'][0]

'https://itunes.apple.com/us/app/kindle-read-ebooks-magazines-textbooks/id302584613?mt=8'

<b>Step3</b> Extract the information for all popular apps.

In [217]:
def get_content(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r)
    des = soup.find_all('div', id = "content")
    apps = soup.find_all("div", class_="lockup product application")
    rate = soup.find_all("div", class_="extra-list customer-ratings")
    dic = []
    global app_name, descript, link, price, category, current_rate, current_count, total_count, total_rate, seller,mul_dev,mul_lang,new_ver_des
    for itm in des:
        try:
            descript = itm.find_all('div',{'class':"product-review"})[0].get_text().strip().split('\n')[2].encode('utf-8')
        except:
            descript = ''
        try:
            new_ver_des = itm.find_all('div',{'class':"product-review"})[1].get_text().strip().split('\n')[2].encode('utf-8')
        except:
            new_ver_des = ''
        try:
            app_name = itm.find_all('div',{'class':"left" })[0].get_text().split('\n')[1]
        except:
            app_name = ''
    for itm in apps:
        category = itm.find_all('span',{'itemprop':"applicationCategory" })[0].get_text()
        price = itm.find_all('div',{'class':"price" })[0].get_text() 
        link = itm.a["href"]
        seller = itm.find_all("span", itemprop="name")[0].get_text()
        try:
            device = itm.find_all("span", itemprop="operatingSystem")[0].get_text()
            if 'and' in device.lower():
                mul_dev = 'Y'
            else:
                mul_dev = "N"
        except:
            mul_dev = "N"
        try:
            lang = itm.find_all("li",class_ = "language")[0].get_text().split(',')
            if len(lang) >1:
                mul_lang = "Y"
            else:
                mul_lang = "N"
        except:
            mul_lang = "N"
    for itm in rate:
        try:
            current_rate = itm.find_all('span',{'itemprop':"ratingValue"})[0].get_text()
        except:
            current_rate = ''
        try:
            current_count = itm.find_all('span',{'itemprop':"reviewCount"})[0].get_text()
        except:
            current_count = ''
        try:
            total_count = itm.find_all('span',{'class':"rating-count"})[1].get_text()
        except:
            try:
                total_count = itm.find_all('span',{'class':"rating-count"})[0].get_text()
            except:
                total_count = ''
        try:
            total_rate = itm.find_all('div', class_="rating",itemprop = False)[0]['aria-label'].split(',')[0]
        except:
            total_rate = ''
    for i in range(3):
        try:
            globals()['user_{0}'.format(i)] = soup.find_all("div", class_="customer-reviews")[0].find_all("span", class_='user-info')[i].get_text().strip( ).split('  ')[-1]
        except:
            globals()['user_{0}'.format(i)] = ''
        try:
            globals()['star_{0}'.format(i)] = soup.find_all("div", class_="customer-reviews")[0].find_all("div", class_="rating")[i]['aria-label']
        except:
            globals()['star_{0}'.format(i)] = ''
        try:
            globals()['comm_{0}'.format(i)] = soup.find_all("div", class_="customer-reviews")[0].find_all("p", class_="content")[i].get_text()
        except:
            globals()['comm_{0}'.format(i)] = ''
       
    dic.append({'app':app_name,'link':link, 'price':price,'category':category,'current rating':current_rate, 
                'current reviews':current_count,'overall rating':total_rate,'overall reviews':total_count,
                'description':descript,'seller':seller,'multiple languages':mul_lang,
                'multiple devices':mul_dev,'new version description':new_ver_des,'user 1':user_0,
                'rate 1':star_0,'comment 1':comm_0,'user 2':user_1,'rate 2':star_1,'comment 2':comm_1,
                'user 3':user_2,'rate 3':star_2,'comment 3':comm_2})
    dic = pd.DataFrame(dic)
    return dic

In [ ]:
full_content = pd.DataFrame()
for itm in app_url['url']:
    content = get_content(itm)
    full_content = full_content.append(content, ignore_index = True)

In [221]:
full_content

,app,category,comment 1,comment 2,comment 3,current rating,current reviews,description,link,multiple devices,...,overall rating,overall reviews,price,rate 1,rate 2,rate 3,seller,user 1,user 2,user 3
0,"Kindle – Read eBooks, Magazines & Textbooks",Books,\n Please focus on reading and listening ex...,\n I can't believe they added a clock in th...,\n You guys did an amazing job updating it ...,2.65152,66 Ratings,Turn your iPhone or iPad into a Kindle with th...,https://itunes.apple.com/us/app/kindle-read-eb...,Y,...,3 and a half stars,245670 Ratings,Free,2 stars,1 star,5 stars,AMZN Mobile LLC,LambdaExpression,Lorring,Prellvis
1,"Audible – audio books, original series & podcasts",Books,\n It's very enjoyable to be able to listen...,\n Works great 99% of the time. When it did...,"\n Audiobooks are a little bit expensive, b...",4.65113,1995 Ratings,"Welcome to Audible. We’re an Amazon company, a...",https://itunes.apple.com/us/app/audible-audio-...,Y,...,4 stars,100917 Ratings,Free,3 stars,5 stars,5 stars,"Audible, Inc.",Smuckitelly,Scorchedterra,vtalbot
2,Wattpad - Free Books and eBook Reader,Books,\n I am absolutely *in love* with Wattpad. ...,\n I love this app sooo much I won't be abl...,\n I love wattpad! I really do! It's my fav...,4.59425,313 Ratings,"Discover Wattpad:At Wattpad, we’re connecting ...",https://itunes.apple.com/us/app/wattpad-free-b...,Y,...,4 and a half stars,236764 Ratings,Free,3 stars,4 stars,5 stars,Wattpad Corp,dg2017xx,RedPandaWorld,Kajdisksn
3,"NOOK - Read Books, Magazines, Newspapers & Comics",Books,\n I would give it 5 stars if the App would...,\n Glad to have Nook on my iPad but wish ma...,\n I love reading books electronically. Hav...,4.33955,1072 Ratings,"Get the FREE NOOK Reading App for your iPad, i...",https://itunes.apple.com/us/app/nook-read-book...,Y,...,4 stars,55593 Ratings,Free,4 stars,3 stars,3 stars,Barnes&Noble,Morgan737364737,Too old to tap,Rstocky7
4,HOOKED - Chat Stories,Books,\n Keeps you on the edge of your seat. If y...,\n It's a great app and I love the chills (...,\n I just got through the first three parte...,4.54263,129 Ratings,HOOKED lets you read amazing chat stories FREE...,https://itunes.apple.com/us/app/hooked-chat-st...,Y,...,4 and a half stars,28004 Ratings,Free,4 stars,2 stars,4 stars,Telepathic Inc.,100% Honest Feedback,Abbidon,Gangrel_Bloodfang
5,OverDrive – Library eBooks and Audiobooks,Books,"\n I don't ever write reviews, but I derive...","\n Love this app. With two kids, this makes...",\n I listen to audiobooks from overdrive on...,4.35417,48 Ratings,"Borrow eBooks, audiobooks, and streaming video...",https://itunes.apple.com/us/app/overdrive-libr...,Y,...,4 stars,63047 Ratings,Free,5 stars,5 stars,3 stars,"OverDrive, Inc.",Blasterbob,"Morbidity, Revisited",Ginastudysmart
6,Goodreads – Book Recommendations and Reviews,Books,\n I like tracking the books I've read. I a...,\n Great app for book readers. Have no comp...,\n I had to use goodreads in a class to kee...,4.51875,160 Ratings,Discover and share books you love on Goodreads...,https://itunes.apple.com/us/app/goodreads-book...,Y,...,4 and a half stars,52969 Ratings,Free,5 stars,4 stars,5 stars,"Goodreads, Inc",Zanykfc,Blueocean7,GotSomeSplaining
7,Google Play Books - Books & Comics,Books,\n This is the WORST Ereader I have ever us...,\n Updated Updated Review: The latest updat...,\n Google Play Books has an incredible sele...,4.53308,1058 Ratings,With Google Play Books for iOS start reading t...,https://itunes.apple.com/us/app/google-play-bo...,Y,...,3 stars,11437 Ratings,Free,1 star,1 star,1 star,"Google, Inc.",attyjames,schoowa,John Hahn
8,"Scribd - Books, audiobooks, magazines, documents",Books,\n Scribd is a fairly good service (now onl...,\n I really like this app but I just don't ...,,3.85714,7 Ratings,"Are you ready for access to the best books, au...",https://itunes.apple.com/us/app/scribd-books-a...,Y,...,4 and a half stars,11990 Ratings,Free,4 stars,4 stars,,Scribd,bfchris,Pata17931,
9,Audiobooks,Book

In [196]:
full_content.to_csv('app.csv',encoding='utf-8',index=True)